In [7]:
import numpy as np
import pandas as pd
import sys, os
from collections import Counter
import rdkit.Chem.rdMolDescriptors
from rdkit import Chem

In [8]:
# import the spreadsheet into a dataframe
file_name='Compound List.xlsx'
df = pd.read_excel(file_name)
df

,name,SMILES,ρ (g/ml),Vm (ml/mol),MW (g/mol)
0,ethanol,CCO,NaN,NaN,NaN
1,benzene,c1ccccc1,NaN,NaN,NaN
2,"2-Isobutyl-4-methyl-1,3-dioxolane",CC1COC(O1)CC(C)C,NaN,NaN,NaN
3,"2-Ethyl-4-methyl-1,3-dioxolane",CCC1OCC(O1)C,NaN,NaN,NaN
4,"3,3-Dimethyloxetane",CC1(COC1)C,NaN,NaN,NaN


In [9]:
# volume increments for the model
Vinc = {
    'H1-0': 0,      # H volume dumped into heavy atoms
    'Al6-3': 7.27,  # This is how RDkit sees Al3-0
    'B3-0': 7.61,
    'B3-1': 22.09,
    'Br1-0': 27.52,
    'C1-0': 14.68,
    'C2-0': 9.89,
    'C2-1': 22.77,
    'C3-0': -0.00,
    'C3-1': 13.23,
    'C3-2': 27.17,
    'C4-0': -8.40,
    'C4-1': 4.46,
    'C4-2': 16.57,
    'C4-3': 29.58,
    'Cl1-0': 24.74,
    'F1-0': 17.63,
    'Ge4-0': 9.36,
    'Ge4-1': 25.92,
    'I1-0': 35.64,
    'N1-0': 14.09,
    'N2-0': 7.42,
    'N2-1': 18.14,
    'N3-0': -3.08,
    'N3-1': 7.74,
    'N3-2': 17.81,
    'O1-0': 14.89,
    'O2-0': 6.25,
    'O2-1': 11.78,
    'P3-0': 10.42,
    'P4-0': -1.94,
    'P4-1': 10.06,
    'Pb4-0': 14.29,
    'R5': 9.41,
    'R6': 6.89,
    'R<5': 10.89,
    'R>6': 3.75,
    'S1-0': 25.92,
    'S2-0': 14.90,
    'S2-1': 26.14,
    'S3-0': 5.58,
    'S4-0': -3.74,
    'Se2-0': 19.00,
    'Si4-0': 9.28,
    'Si4-1': 23.35,
    'Sn4-0': 14.47,
    'Ti4-0': 6.09,
    'aromat': 1.82
}

In [10]:
# Set of parameters fitted using <10 entries 
rares = set(('Al6-3', 'B3-1', 'C1-0', 'Ge4-1', 'N2-1', 'P4-1', 'Pb4-0', 'S3-0', 'Se2-0', 'Ti4-0'))

def GetARH(atom):
    """ return an atom code consistent with the keys of the 'Vinc' dictionary """ 
    # be careful to take into account nD = number of deuterium atoms !
    nD = len([x for x in atom.GetNeighbors() if x.GetMass()>2 and x.GetSymbol()=='H'])
    if nD > 0:
        print('nD %u %s' %(nD, arg))
    return atom.GetSymbol() + str(atom.GetTotalDegree()) + '-' + str(atom.GetTotalNumHs()+nD)

def areAromatic(atoms):
    """ test whether all input atoms (typically from a ring) are aromatic """
    for at in atoms:
        if not at.GetIsAromatic():
            return False
    return True

def setRingData(dic, mol, maxi=6, mini=5):
    """ add keys relative to rings in dictionary 'dic' for molecule 'mol' """
    ri = mol.GetRingInfo()
    for ring in ri.AtomRings():
        size = len(ring)
        if size > maxi:
            label = 'R>'+str(maxi)
        elif size < mini:
            label = 'R<'+str(mini)
        else:
            label = 'R%u' %len(ring)
        dic[label] += 1
        # contribute also +1 aromatic ring ?
        atoms = [mol.GetAtomWithIdx(i) for i in ring]
        if areAromatic(atoms):
            dic['aromat'] += 1
    return dic

In [11]:
for i in range(len(df['SMILES'])):
    arg = df['SMILES'][i]
    x = arg
    def GetMol(arg):
        """ return RDkit molecule from input argument: either .mol file or SMILES """
        if not os.path.isfile('arg'):
            try:
                return Chem.MolFromSmiles(x)
            except ValueError:
                return None
        if arg.endswith('.mol'):
            return Chem.MolFromMolFile(arg)
        return None

    if len(sys.argv)<2:
        print(sys.argv[0]+' <molecules as MDF .mol files or quoted SMILES strings>')
        sys.exit()
    for arg in sys.argv[1:]:            # iterate over all input arguments
        mol = GetMol(arg)               # each one is for a molecule
        if mol is None:
            print('Error: compound %s cannot be read' %arg)
            continue                    # skip spurious input strings
        mf = Counter([a.GetSymbol() for a in Chem.AddHs(mol).GetAtoms()])  # empirical formula
        if mf['C'] == 0:
            print('Warning: %s not in applicability domain (no C atom)' %arg)
        if mf['H'] < 2:
            print('Warning: %s exhibits %u H atom (low accuracy expected)' %(arg, mf['H']))
        dic = Counter()                 # counts number of fragments/rings
        for at in mol.GetAtoms():
            k = GetARH(at)              # 'k' defines the kind of fragment associated with each non-H atom
            dic[k] +=1                  # the number of such fragments is incremented
        setRingData(dic, mol)           # add the number of aromatic rings to 'dic'
        dangerous = set(dic.keys()) & rares  # do we use parameters adjusted using <10 entries ?
        if dangerous:
            print('Warning: possibly ill-defined parameters for %s: ' %arg + ' '.join(sorted(list(dangerous))))
        try:
            Vm = sum([dic[k]*Vinc[k] for k in dic])  # adds increments involved
        except KeyError:                # 'k' not found in Vinc for unknown fragments
            print(dic)
            print('Error: missing parameter(s) for compound: ' +arg)        # nevertheless print a line in output file
            continue                    # skip this compound
        if Vm == 0:                     # compound made only of H/D atoms
            print('Error: cannot calculate molar volume for compound %s' %arg)
            continue
        Mw = rdkit.Chem.rdMolDescriptors.CalcExactMolWt(mol)  # molecular weight needed...
        calcDensity = Mw/Vm                                   # to convert molar volume to density
    
    df['ρ (g/ml)'][i], df['MW (g/mol)'][i], df['Vm (ml/mol)'][i] = round(calcDensity, 4), round(Mw, 3), Vm
df

/Users/junqingzhu/opt/anaconda3/envs/combustion/lib/python3.7/site-packages/ipykernel_launcher.py:48: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


,name,SMILES,ρ (g/ml),Vm (ml/mol),MW (g/mol)
0,ethanol,CCO,0.7948,57.93,46.042
1,benzene,c1ccccc1,0.8860,88.09,78.047
2,"2-Isobutyl-4-methyl-1,3-dioxolane",CC1COC(O1)CC(C)C,0.9169,157.17,144.115
3,"2-Ethyl-4-methyl-1,3-dioxolane",CCC1OCC(O1)C,0.9428,123.13,116.084
4,"3,3-Dimethyloxetane",CC1(COC1)C,0.8519,101.04,86.073


In [12]:
# as a sanity check, the predicted density of ethanol and benzene should be 0.7948 g/ml and 0.8860 g/ml, respectively
# export the dataframe back into the spreadsheet
df.to_excel('./Compound List.xlsx', index = False)